In [1]:
!pip install litserve

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.4/341.4 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 425.7/425.7 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.1/164.1 kB 6.5 MB/s eta 0:00:00


In [9]:
import requests
import torch
from PIL import Image
from transformers import VisionEncoderDecoderModel, ViTImageProcessor, GPT2TokenizerFast
import urllib.parse as parse
import os

# Verify URL function
def check_url(string):
    try:
        result = parse.urlparse(string)
        return all([result.scheme, result.netloc, result.path])
    except:
        return False

# Load an image from a URL or local path
def load_image(image_path):
    if check_url(image_path):
        return Image.open(requests.get(image_path, stream=True).raw)
    elif os.path.exists(image_path):
        return Image.open(image_path)
    else:
        raise ValueError(f"Invalid image path: {image_path}")

# HuggingFace API class for image captioning with accelerator support
class ImageCaptioningLitAPI:
    def setup(self, **kwargs):
        # Get parameters from kwargs with default values
        accelerator = kwargs.get('accelerator', 'auto')
        devices = kwargs.get('devices', 1)
        workers_per_device = kwargs.get('workers_per_device', 1)

        # Choose the device based on the accelerator input
        if accelerator == "cuda" and torch.cuda.is_available():
            self.device = "cuda"
        else:
            self.device = "cpu"

        # If accelerator="auto", auto-detect GPU or fallback to CPU
        if accelerator == "auto":
            self.device = "cuda" if torch.cuda.is_available() else "cpu"

        print(f"Using device: {self.device}")

        # Load the ViT Encoder-Decoder Model
        model_name = "nlpconnect/vit-gpt2-image-captioning"
        self.model = VisionEncoderDecoderModel.from_pretrained(model_name).to(self.device)

        # Load the corresponding Tokenizer
        self.tokenizer = GPT2TokenizerFast.from_pretrained(model_name)

        # Load the Image Processor
        self.image_processor = ViTImageProcessor.from_pretrained(model_name)

    # Generate image caption
    def predict(self, image_path):
        image = load_image(image_path)

        # Preprocessing the Image
        img = self.image_processor(image, return_tensors="pt").to(self.device)

        # Generating captions
        output = self.model.generate(**img)

        # Decode the output to generate the caption
        caption = self.tokenizer.batch_decode(output, skip_special_tokens=True)[0]

        return caption

# Instantiate the class and set up
api = ImageCaptioningLitAPI()

# Pass the accelerator and other parameters here
api.setup(accelerator="cuda", devices=1, workers_per_device=1)


Using device: cuda
Generated Caption: a man in a suit and hat standing in front of a building 


In [13]:
# Example 1
image_path = "/content/cbimage.png"

caption = api.predict(image_path)
print(f"Generated Caption: {caption}")

Generated Caption: a man in a suit and tie holding a red and white flag 


In [16]:
# Example 3
image_path = "/content/image (2).png"

caption = api.predict(image_path)
print(f"Generated Caption: {caption}")

Generated Caption: a view from a boat of a beach with a large body of water 
